Discrete Fourier transform (DFT)

$\Rightarrow$ Transform the sequence $\{a_0, a_1, \dots, a_{N-1}\}$ into $\{\phi_0, \phi_1, \dots, \phi_{N-1}\}$, where $\phi_k = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} a_j e^{2\pi i \left( \frac{jk}{N} \right)}$

In [ ]:
import numpy as np

def DFT(seq):
    N = len(seq)
    X = np.zeros(N, dtype=complex)
    for k in range(N):
        for j in range(N):
            X[k] += seq[j]*np.exp(2j*np.pi*(k*j/N))
    return X/np.sqrt(N)

Quantum Fourier transform(QFT)

Let $N=2^{n}$.

$\Rightarrow$ Transform $\left|\psi\right> = a_{0} \left| 0 \right> + \cdots + a_{N-1} \left| N-1\right>$
into $\left|\phi\right> = \phi_{0} \left| 0 \right> + \cdots + \phi_{N-1} \left| N-1\right>$.

$\Rightarrow \left| j \right> \xrightarrow{\text{QFT}} \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} e^{2\pi i \left(\frac{jk}{N}\right)} \left| k \right>$

$\Rightarrow \left| j_{n-1} \right> \left| j_{n-2} \right> \cdots \left| j_{1} \right> \left| j_{0} \right> \xrightarrow{\text{QFT}}$  
$\qquad \frac{1}{\sqrt{2}} \left( \left|0\right> + e^{2\pi i \left(\frac{j_0}{2}\right)} \left|1\right> \right) \otimes
\frac{1}{\sqrt{2}} \left( \left|0\right> + e^{2\pi i \left(\frac{j_1}{2} + \frac{j_0}{2^2}\right)} \left|1\right> \right) \otimes \cdots$  
$\qquad\qquad \otimes \frac{1}{\sqrt{2}} \left( \left|0\right> + e^{2\pi i \left( \frac{j_{n-2}}{2} + \cdots + \frac{j_1}{2^{n-2}} + \frac{j_0}{2^{n-1}} \right)} \left|1\right> \right) \otimes
\frac{1}{\sqrt{2}} \left( \left|0\right> + e^{2\pi i \left( \frac{j_{n-1}}{2} + \frac{j_{n-2}}{2^2} + \cdots + \frac{j_1}{2^{n-1}} + \frac{j_0}{2^n} \right)} \left|1\right> \right)$

Note: phaseshift operator
$R_{\theta}(\theta) = 
\begin{bmatrix}
1 & 0 \\
0 & e^{i\theta}
\end{bmatrix}$.

In [ ]:
import pennylane as qml
import matplotlib.pyplot as plt

n_qubits = 6

dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def QFT(seq):
    qml.MottonenStatePreparation(seq, wires=[i for i in range(n_qubits)])
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        for j in range(i+1, n_qubits):
            qml.ControlledPhaseShift(np.pi/2**(j-i), wires=[j, i])
    for i in range(n_qubits//2):
        qml.SWAP(wires=[i, n_qubits-i-1])
    return qml.state()

# QFT circuit
test_seq = np.array([1 for i in range(2**n_qubits)])
test_seq = test_seq/np.linalg.norm(test_seq)
fig, ax = qml.draw_mpl(QFT)(test_seq)
plt.show()

In [ ]:
# Example
N = 2**n_qubits
t = np.arange(N)
f = 4
# {a_{0}, ..., a_{N-1}}
seq = np.sin(2*np.pi*(f*t/N))
init_seq = seq/np.linalg.norm(seq)

# initial sequence plot
plt.plot(t, init_seq)
plt.show()

In [ ]:
# results
DFT_seq = DFT(init_seq)    
QFT_seq = QFT(init_seq)

print('fidelity:', abs(np.dot(DFT_seq, QFT_seq)))

fig, axs = plt.subplots(2, 1, figsize=(10, 8))

axs[0].stem(np.abs(DFT_seq), markerfmt='ro', linefmt='None')
axs[0].set_title("DFT")
axs[1].stem(np.abs(QFT_seq), markerfmt='bo', linefmt='None')
axs[1].set_title("QFT")

plt.show()